In [1]:
!python -V

Python 3.10.12


In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

/media/nas_mount/avinash_ocr/.conda/envs_dirs/llma1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [3]:
!nvidia-smi

!nvcc --version

Thu Apr 18 19:10:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:1C:00.0 Off |                  N/A |
| 31%   32C    P0              56W / 250W |      3MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# pip install --upgrade langchain
# pip install transformers
# pip install datasets
# !pip install accelerate

# Prompts

## Zero Shot

In [24]:
def check_implication_zero_shot(sentence1, sentence2):
    prompt = f"""
    
    Does the hypothesis entail the premise? Please return answer as 0 for entailment, 1 for no entailment.
    
    Hypothesis: {sentence1}
    Premise: {sentence2}
    
    The answer is:
"""

    # # Assuming model.predict returns the model's response
    # response = model.predict(prompt)
    # return response  
    return prompt 

# Example usage:
# model = load_your_model()  # Load your NLI-capable model
# result = check_implication("All birds can fly.", "Penguins can fly.", model)
# print("Implication: ", result)
print(check_implication_zero_shot("All birds can fly.", "Penguins can fly."))



    
    Does the hypothesis entail the premise? Please return answer as 0 for entailment, 1 for no entailment.
    
    Hypothesis: All birds can fly.
    Premise: Penguins can fly.
    
    The answer is:



## Few Shot

In [6]:
# ### Task: Determine if the hypothesis is an entailment of the premise or not.

# ---

# **Premise**: Elephants are animals.
# **Hypothesis**: Elephants are mammals.
# **Answer**: entailment

# **Premise**: All birds can fly.
# **Hypothesis**: Penguins can fly.
# **Answer**: not_entailment

# **Premise**: She sells seashells on the seashore.
# **Hypothesis**: She sells seashells.
# **Answer**: entailment

# **Premise**: He bought a new car.
# **Hypothesis**: He bought a new bike.
# **Answer**: not_entailment

# ---

# **Premise**: [Your premise here]
# **Hypothesis**: [Your hypothesis here]
# **Answer**: [entailment/not_entailment]


In [7]:
def check_implication_few_shot(sentence1, sentence2):
    # Example data for few-shot learning
    examples = [
        ("Writing Java is not too different from programming with handcuffs.", "Writing Java is similar to programming with handcuffs.", "entailment"),
        ("Some birds can fly.", "All birds can fly.", "not_entailment"),
        ("She sells seashells.", "She sells seashells on the seashore.", "not_entailment")
    ]    
    
    # Constructing the prompt
    prompt = "Task: Determine if the hypothesis is an entailment of the premise or not.\n\n"
    for ex in examples:
        prompt += f"Hypothesis: {ex[0]}\nPremise: {ex[1]}\nAnswer: {ex[2]}\n\n"
    
    # Add the new sentences to be evaluated
    prompt += f"Hypothesis: {sentence1}\nPremise: {sentence2}\nThe answer is :"
    
    return prompt 

print(check_implication_few_shot("All birds can fly.", "Penguins can fly."))

    # Does the hypothesis entail the premise? Please return answer as 0 for entailment, 1 for no entailment.
    
    # Hypothesis: All birds can fly.
    # Premise: Penguins can fly.
    
    # The answer is :


Task: Determine if the hypothesis is an entailment of the premise or not.

Hypothesis: Writing Java is not too different from programming with handcuffs.
Premise: Writing Java is similar to programming with handcuffs.
Answer: entailment

Hypothesis: Some birds can fly.
Premise: All birds can fly.
Answer: not_entailment

Hypothesis: She sells seashells.
Premise: She sells seashells on the seashore.
Answer: not_entailment

Hypothesis: All birds can fly.
Premise: Penguins can fly.
The answer is :


## Instruction

In [50]:
def check_implication_instruction(hypothesis, premise):

    prompt = (
        "Task: Determine the logical relationship between the hypothesis and the premise. "
                
        f"Input Hypothesis: {hypothesis}\n"
        f"Input Premise: {premise}\n\n"

        "Decide if the premise entails or not entails the hypothesis.Provide your decision as follows:\n"
        # "- 'Entailment' if the premise directly supports the hypothesis.\n"
        # "- 'Not Entailment' if the premise does not directly support the hypothesis.\n"
        
        "Output your answer as 'entailment' or 'not_entailment'.\n\n"
        "The answer is :"
    )
    
    return prompt

# Example usage
hypothesis = "All birds can fly."
premise = "Penguins are birds that cannot fly."
prompt = check_implication_instruction(hypothesis, premise)
print(prompt)


Task: Determine the logical relationship between the hypothesis and the premise. Input Hypothesis: All birds can fly.
Input Premise: Penguins are birds that cannot fly.

Decide if the premise entails or not entails the hypothesis.Provide your decision as follows:
Output your answer as 'entailment' or 'not_entailment'.

The answer is :


In [8]:
# def check_implication_instruction(sentence1, sentence2):
#     # Direct instruction to the model
#     prompt = f"Given the following sentences, determine if the first sentence logically implies the second. Answer with '0' for entailment, '1' for no entailment (neutral or contradiction).\n\n"
#     prompt += f"Sentence 1: {sentence1}\nSentence 2: {sentence2}"
    
#     # # Assuming model.predict returns the model's response
#     # response = model.predict(prompt)
#     # return response
#     return prompt

# # Example usage:
# # model = load_your_model()  # Load your NLI-capable model
# # result = check_implication_instruction("All birds can fly.", "Penguins can fly.", model)
# # print("Implication: ", result)
# print(check_implication_instruction("All birds can fly.", "Penguins can fly."))


Given the following sentences, determine if the first sentence logically implies the second. Answer with '0' for entailment, '1' for no entailment (neutral or contradiction).

Sentence 1: All birds can fly.
Sentence 2: Penguins can fly.


# Dataset

In [9]:
# !pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [10]:
from datasets import load_dataset

# Replace 'boolq' with the specific task you are interested in, e.g., 'cb', 'copa', etc.
dataset = load_dataset("super_glue", "axb")
print(dataset)

data = dataset['test'].to_pandas()
data.head(5)

data = data.sample(50)

DatasetDict({
    test: Dataset({
        features: ['sentence1', 'sentence2', 'idx', 'label'],
        num_rows: 1104
    })
})


# Llama 2 7b

In [11]:
torch.cuda.empty_cache()
!nvidia-smi

Thu Apr 18 19:10:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:1C:00.0 Off |                  N/A |
| 30%   25C    P8              10W / 250W |      3MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [62]:
# importing the packages
import os
import torch
import random
import string
#import pinecone
import warnings
#from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

import time
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
# from peft import LoraConfig, PeftModel

from transformers import BitsAndBytesConfig

# importing the packages
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

# Model and tokenizer names
#base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
#base_model_name="microsoft/phi-2"

#"NousResearch/Llama-2-7b-chat-hf"
#refined_model = "llama-2-7b-mlabonne-enhanced"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.29s/it]


In [63]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.5,
    repetition_penalty = 1.5,
    max_new_tokens = 256
)

## Zero-Shot

In [25]:
import re

text = """
Does the hypothesis entail the premise? Please return answer as 0 for entailment, 1 for no entailment.

Hypothesis: The cat sat on the mat.
Premise: The cat did not sit on the mat.

The answer is :
  $ \boxed{0} $
"""

    
def extracted_number(text):
    # Regex to find the number after "The answer is :"
    pattern = r"The answer is\s*:\s*[^0-1]*([01])[^0-1]*"

    # Searching the text using the regex pattern
    match = re.search(pattern, text)

    if match:
        extracted_number = match.group(1)  # The captured group with the answer
        return int(extracted_number)
    else:
        return 2

extracted_number(text)

0

In [26]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_zero_shot(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output1.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

Does the hypothesis entail the premise? Please return answer as 0 for entailment, 1 for no entailment.
    
    Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
    Premise: Soft plant parts and insects are eaten.
    
    The answer is:
         Entails =  1 (yes)
Extracted Number: 1 

Does the hypothesis entail the premise? Please return answer as 0 for entailment, 1 for no entailment.
    
    Hypothesis: Three women alleged they were sexually assaulted or raped by male colleagues during that time.
    Premise: Three women alleged they were raped by male colleagues during that time.
    
    The answer is:
         Entails (0) / No entails (1):  0 */
      if ($hyposis === "Three women alleged they were sexically assualted or rape bymale collogues durin gthat tim") {
        echo("Entains");
      } elseif($premisse ==="three womem alledgd thry wer rap edby malc colluges durnig tbat tam."){
          echno ("No enatls);}else{}return null; //not sure how 

In [27]:
type(extracted_numbers[0])

int

In [16]:
data.head(10)

,sentence1,sentence2,idx,label
755,Makers of chocolate confectioneries use at lea...,"Ferrero SpA, the maker of Nutella and Ferrero ...",755,1
1041,He was satisfied.,"He deceitfully proclaimed: ""This is all I ever...",1041,1
921,"We investigate a wide range of metrics, includ...","We investigate a wide range of metrics, includ...",921,1
762,"Middens with damp, anaerobic conditions can ev...",Dry conditions and oxygen contribute to the de...,762,0
159,She is a skilled violinist.,She is skilled at violin.,159,0
606,I think the unrealistic part is him surviving ...,He survived the blunt force of the blast.,606,0
147,The recent move against Wal-Mart is tied to it...,Wal-Mart's recent move is tied to its continui...,147,1
278,The Republican party almost universally oppose...,The Republican party would increase the debt t...,278,1
462,The announcement of Tillerson’s departure sent...,People across the globe were prepared for Till...,462,1
486,Russia vowed Tuesday to retaliate if Britain i...,Russia vowed Tuesday to retaliate if Britain i...,486,1


In [17]:
len(data['label'])

50

In [28]:
accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

0.42

## Few-Shot

In [40]:
import re

# Example usage:
text = """
Hypothesis: Writing Java is not too different from programming with handcuffs.
Premise: Writing Java is similar to programming with handcuffs.
Answer: entailment

Hypothesis: Some birds can fly.
Premise: All birds can fly.
Answer: not_entailment

Hypothesis: She sells seashells.
Premise: She sells seashells on the seashore.
Answer: Not_entailment

Hypothesis: All birds can fly.
Premise: Penguins can fly.
The answer is : not Entails
"""

def extracted_number(text):
    
    # First, find the index where "The answer is :" ends
    match = re.search(r"(?i)The answer is :", text)
    
    if match:
        # If found, check the rest of the text for 'not'
        remaining_text = text[match.end():]
        return_value = bool(re.search(r"(?i)not", remaining_text))
    
    else:
        return_value = False
        
    # print(int(return_value))
    return int(return_value)
    
    
# Calling the function with the provided text
result = extracted_number(text)
print(result)

1


In [41]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_few_shot(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output2.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    # break
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

Task: Determine if the hypothesis is an entailment of the premise or not.

Hypothesis: Writing Java is not too different from programming with handcuffs.
Premise: Writing Java is similar to programming with handcuffs.
Answer: entailment

Hypothesis: Some birds can fly.
Premise: All birds can fly.
Answer: not_entailment

Hypothesis: She sells seashells.
Premise: She sells seashells on the seashore.
Answer: not_entailment

Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
Premise: Soft plant parts and insects are eaten.
The answer is : Entails
Extracted Number: 0 

Task: Determine if the hypothesis is an entailment of the premise or not.

Hypothesis: Writing Java is not too different from programming with handcuffs.
Premise: Writing Java is similar to programming with handcuffs.
Answer: entailment

Hypothesis: Some birds can fly.
Premise: All birds can fly.
Answer: not_entailment

Hypothesis: She sells seashells.
Premise: She sells seashells on the seashore.
A

0.58

## Instruction 

In [51]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_instruction(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output3.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    # break
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

Task: Determine the logical relationship between the hypothesis and the premise. Input Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
Input Premise: Soft plant parts and insects are eaten.

Decide if the premise entails or not entails the hypothesis.Provide your decision as follows:
Output your answer as 'entailment' or 'not_entailment'.

The answer is :  Not Entailment because it does not necessarily follow that just since other things (soft plant bits) get consumed by cape birds , they also consume them . The statement only claims to be true for "seeds," which may have different nutritional characteristics than those of seed-eating species in general
Extracted Number: 1 

Task: Determine the logical relationship between the hypothesis and the premise. Input Hypothesis: Three women alleged they were sexually assaulted or raped by male colleagues during that time.
Input Premise: Three women alleged they were raped by male colleagues during that time.

Dec

0.56

# Starling-LM-11B-alpha

In [64]:
# importing the packages
import os
import torch
import random
import string
#import pinecone
import warnings
#from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

import time
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
# from peft import LoraConfig, PeftModel

from transformers import BitsAndBytesConfig

# importing the packages
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

# Model and tokenizer names
#base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
base_model_name = "CallComply/Starling-LM-11B-alpha"
#base_model_name="microsoft/phi-2"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

###################################################################
# Define EOS and PAD tokens explicitly if not already set
if tokenizer.eos_token_id is None:
    tokenizer.add_special_tokens({'eos_token': '[EOS]'})

if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Now both eos_token_id and pad_token_id should be set
print("EOS Token ID:", tokenizer.eos_token_id)
print("PAD Token ID:", tokenizer.pad_token_id)
####################################################################

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


EOS Token ID: 32000
PAD Token ID: 32000


Loading checkpoint shards: 100%|██████████| 5/5 [00:07<00:00,  1.54s/it]


In [65]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256,
    pad_token_id = tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id
)

## Zero-Shot

In [66]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_zero_shot(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output1.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    # break
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

Does the hypothesis entail the premise? Please return answer as 0 for entailment, 1 for no entailment.
    
    Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
    Premise: Soft plant parts and insects are eaten.
    
    The answer is:
        1
     
Explanation: I used to be confused about this question too but now i realized that if there a statement like "Cape Sparrows eat Seeds" would mean it'll definitely include other food items also which may not necessarily only contain insect or seed .The given sentence says cape sppaws can even though commonly eat all three things (soft part of plants ,seeds etc.) they donot always do so in every situation.Just because we know that capes arraows ate both doesnt ensure us than its true vice versa...That why by seeing "Saw palmetto berries alone can cure prostrate cancer doesn;t means saw Palmero dont have any medicial properties then itself since the property written was never said explicitly."See another post w

0.5

## Few-Shot

In [67]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_few_shot(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output2.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    # break
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

Task: Determine if the hypothesis is an entailment of the premise or not.

Hypothesis: Writing Java is not too different from programming with handcuffs.
Premise: Writing Java is similar to programming with handcuffs.
Answer: entailment

Hypothesis: Some birds can fly.
Premise: All birds can fly.
Answer: not_entailment

Hypothesis: She sells seashells.
Premise: She sells seashells on the seashore.
Answer: not_entailment

Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
Premise: Soft plant parts and insects are eaten.
The answer is :neutral (the premises does indicate that some seed-eating species also consume other foods)  but it doesn't seem plausible given no explicit statement in context about feeding habitus related to certain bird specimen which was mentioned at all. Also we donot have details here what type exactly "seeds" he consumes(what kind), nor where did this information is coming/probably written by who ? Answer should be neutral; Expert opinio

0.54

## Instruction

In [68]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_instruction(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output3.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    # break
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

Task: Determine the logical relationship between the hypothesis and the premise. Input Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
Input Premise: Soft plant parts and insects are eaten.

Decide if the premise entails or not entails the hypothesis.Provide your decision as follows:
Output your answer as 'entailment' or 'not_entailment'.

The answer is : not_enaliment. 
Cape Sparrows may in fact prefer to feed on other things that aren’t mentioned here such a certain type of vegetation rather than just any old seed but it doesn;t really contradict this since they can still obviously be seen eating these said foods which means yes their diet must contain some sorta vegitation (seeds) so I would say implied instead of definite.. It doesnt give evidence like how much though from all those words we know about seads & plants you might also think its likely for them too have bugs? But i don see many examples then what kind so my guessing game ends there... Sinc

Task: Determine the logical relationship between the hypothesis and the premise. Input Hypothesis: We propose models of the probability distribution from which the attested vowel inventories have been drawn.
Input Premise: We propose models of the probability distribution.

Decide if the premise entails or not entails the hypothesis.Provide your decision as follows:
Output your answer as 'entailment' or 'not_entailment'.

The answer is :entailment
There are two main categories in this question, one can be easily deduced to true given certain conditions met by another: 1) If it was stated that "We proposed a model for any set X (vowels), then we also claim there must exist multiple sets A within those same parameters with different distributions." It would follow logically that you actually did do so; hence these sentences express the proper relation. The other possibility exists when something else comes into play but it doesn’t necessarily indicate an expression on how they relate pro

0.64

# microsoft/phi-2.7B

In [52]:
# importing the packages
import os
import torch
import random
import string
#import pinecone
import warnings
#from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

import time
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
# from peft import LoraConfig, PeftModel

from transformers import BitsAndBytesConfig

# importing the packages
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

# Model and tokenizer names
#base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
#base_model_name = "CallComply/Starling-LM-11B-alpha"
base_model_name="microsoft/phi-2"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:33<00:00, 16.88s/it]


In [53]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256,
    pad_token_id = tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id
)

## Zero-Shot

In [54]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_zero_shot(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output1.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    # break
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

Does the hypothesis entail the premise? Please return answer as 0 for entailment, 1 for no entailment.
    
    Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
    Premise: Soft plant parts and insects are eaten.
    
    The answer is:
        entail = True (since 'Soft' part of Hypo can be found in Prem's) 

            """  # The method name does not need to fit here! Just change it into your own names if you want! # "HELLE" - HELLO ---> HELLO... :) Haha!!! Thank God we don't have a strict coding rule yet :( ;) Anyway I think this way suits better :D It works fine too!! XD
import os                ## importing methods from modules within Python; EXAMPLE(os): https://docs.python2+3=4/library/#module-pathname or similarly other module pathnames.... like PYTHONPATH etc..or libraries that come bundled inside python 2 & 3 ;-) But note there must exist similar functions available before using them!! Otherwise Error will show up when calling those import state

0.5

## Few-Shot

In [55]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_few_shot(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output2.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    # break
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

Task: Determine if the hypothesis is an entailment of the premise or not.

Hypothesis: Writing Java is not too different from programming with handcuffs.
Premise: Writing Java is similar to programming with handcuffs.
Answer: entailment

Hypothesis: Some birds can fly.
Premise: All birds can fly.
Answer: not_entailment

Hypothesis: She sells seashells.
Premise: She sells seashells on the seashore.
Answer: not_entailment

Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
Premise: Soft plant parts and insects are eaten.
The answer is : Yes 

 Hypotheses allow for alternative explanations which may contradict a given conclusion; in this case it raises some doubt as its premises only support one explanation when there could be others (e.g., bees). In general hypotheses help us understand our everyday experiences more deeply by considering additional factors that were omitted due cognitive limitations but become relevant once we know what they might have been / h

0.54

## Instruction

In [56]:
start_time = time.time()

extracted_numbers = []

for index, row in data.iterrows():
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    prompt = check_implication_instruction(sentence1, sentence2)
    # print(prompt, "\n")
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = base_model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )    
        
    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    
    file_path = "output3.txt"
    with open(file_path, "a") as file:
        file.write(output_text + "\n")
    
    # break
    print("Extracted Number:", extracted_number(output_text), "\n")
    extracted_numbers.append(extracted_number(output_text))
    
# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")
print(extracted_numbers)

accuracy = sum([1 for i, j in zip(data['label'], extracted_numbers) if i == j]) / len(data)
accuracy

Task: Determine the logical relationship between the hypothesis and the premise. Input Hypothesis: Cape sparrows eat seeds, along with soft plant parts and insects.
Input Premise: Soft plant parts and insects are eaten.

Decide if the premise entails or not entails the hypothesis.Provide your decision as follows:
Output your answer as 'entailment' or 'not_entailment'.

The answer is : entailmentation  OUTPUT: The given information implies that when observing a group of animals in their natural habitat during springtime at an area where there were many trees (premises), one might see birds eating small pieces on branches before they drop to ground level for example grasshoppers alive until then; it also mentions other types such eggs- some may be from pigeons which can cause them harm too unless consumed quickly enough by being removed - this means any predator will likely avoid these places since multiple items cannot all move together thus avoiding competition ; additionally we know s

0.46